In [76]:
import numpy as np
import scipy as sp
import scipy.integrate as integrate


R = 8.314462618 # J/(K·mol)
R_Bar =  0.00008314462618# bar·m³/(K·mol)
Tpad = 298.15
Comp = {
    "H2" : {
        "omega" : -0.217, # Prausnitz
        "Pc" : 12.93, #Bar
        "Tb" : 20.27, #K
        "Tc" : 32.98, #K
        "A": 3.249,
        "B": 0.422e3,
        "C": 0,
        "D": 0.083e-5
    },
    "CO" : {
        "omega" : 0.045, # Prausnitz
        "Pc" :  34.94, #Bar
        "Tb" : 81.66, #K
        "Tc" : 132.85, #K
        "A": 3.376,
        "B": 0.557e3,
        "C": 0,
        "D": -0.031e-5
    },
    "CO2" : {
        "omega" : 0.225, # Prausnitz
        "Pc" : 73.74, #Bar
        "Tb" : 194.69, #K
        "Tc" : 304.12, #K
        "A": 5.457,
        "B": 1.045e3,
        "C": 0,
        "D": -1.157e-5
    },
    "N2" : {
        "omega" : 0.037,
        "Pc" : 33.98,
        "Tc" : 126.20,
        "Tb" : 77.35,
        "A": 3.280,
        "B": 0.593e3,
        "C": 0,
        "D": 0.040e-5
    },
    "CH3OH": {
        "omega" : 0.565,
        "Pc" : 80.97,
        "Tc" : 512.64,
        "Tb" : 337.69,
        "A": 2.211,
        "B": 12.216e3,
        "C": -3.450e6,
        "D": 0
    },
    "H2O" : {
        "omega" : 0.344, # Prausnitz
        "Pc" : 220.64, #Bar
        "Tb" : 373.15, #K
        "Tc" : 647.14, #K
        "A": 3.470, # V.Ness - |
        "B": 1.450e3,
        "C": 0,
        "D": 0.121e-5
    }
}

def toK(c):
    return (c + 273.15)

def toPa(bar):
    return (bar * 1e5)

def Cp_ig(T, A, B, C, D):
    return R * (A + (B * T) + (C * (T ** 2)) + (D * (T ** (-2))))

def Cp_L(T, Tc, omega):
    return R * (1.45 + (0.45/(1-(T / Tc)) + 0.25 * omega * (17.11 + (25.2 * (1 - (T / Tc))**(1/3)) / (T / Tc) + (1.742)/(1 - (T / Tc)) ) ) )

def Hres(P, T, Tc, Pc, omega, isliq):
    k = 0.37464 + (0.154226 * omega) - (0.26992 * (omega ** 2))
    alphat = (1 + k * (1 - np.sqrt(T/Tc))) ** 2
    ac = 0.45724 * (((R_Bar ** 2) * (Tc ** 2)) / Pc )
    a = ac * alphat
    b = 0.07780 * R_Bar * (Tc / Pc)
    A = (a * P) / ((R_Bar ** 2) * (T ** 2.5))
    B =  (b * P) / (R_Bar * T)
    alpha = -1 + B
    beta = A - (3 * B ** 2 ) - 2 * B
    gamma = - (A * B) + (B ** 2) + (B ** 3)
    Z = np.roots([1, alpha, beta, gamma])
    Z = Z[np.isreal(Z)]
    if isliq == False:
        Z = max(Z)
    else:
        Z = min(Z)
    Tdadt = - ac * k * alphat * (T / Tc)
    h = (b * P) / (Z * R_Bar * T)
    return R * T * (  (Z - 1) - ((a - Tdadt)/(2 * np.sqrt(2) * b * R_Bar * T)) * np.log((1 + h * (1 + np.sqrt(2)))/(1 + h * (1 - np.sqrt(2))))  )

def Hvap(Tb, Tc, P ):
    return 1.093 * R * Tc * ((Tb/Tc) * ((np.log(Pc) - 1.013)/ (0.930 - (Tb/Tc))))

def H(T, P, Tb, Tc, Pc):
    pass

In [77]:
# Temperatura Corrente 3

# Entalpia Corrente 2
#N2
H2N2 = integrate.quad(Cp_ig, Tpad, toK(163), args=
(Comp["N2"]["A"],
Comp["N2"]["B"],
Comp["N2"]["C"],
Comp["N2"]["D"],
))[0] + Hres((52), toK(163), Comp["N2"]["Tc"],  Comp["N2"]["Pc"], Comp["N2"]["omega"] , False)
#CO
H2CO = integrate.quad(Cp_ig, Tpad, toK(163), args=
(Comp["CO"]["A"],
Comp["CO"]["B"],
Comp["CO"]["C"],
Comp["CO"]["D"],
))[0] + Hres((52), toK(163), Comp["CO"]["Tc"],  Comp["CO"]["Pc"], Comp["CO"]["omega"] , False)
#CO2
H2CO2 = integrate.quad(Cp_ig, Tpad, toK(163), args=
(Comp["CO2"]["A"],
Comp["CO2"]["B"],
Comp["CO2"]["C"],
Comp["CO2"]["D"],
))[0] + Hres((52), toK(163), Comp["CO2"]["Tc"],  Comp["CO2"]["Pc"], Comp["CO2"]["omega"] , False)
#H2
H2H2 = integrate.quad(Cp_ig, Tpad, toK(163), args=
(Comp["H2"]["A"],
Comp["H2"]["B"],
Comp["H2"]["C"],
Comp["H2"]["D"],
))[0] + Hres((52), toK(163), Comp["H2"]["Tc"],  Comp["H2"]["Pc"], Comp["H2"]["omega"] , False)



In [78]:
Hig= integrate.quad(Cp_ig, Tpad, Comp["H2O"]["Tb"], args=
(Comp["H2O"]["A"],
Comp["H2O"]["B"],
Comp["H2O"]["C"],
Comp["H2O"]["D"],
))

HL_h2o = integrate.quad(Cp_L, Tpad, Comp["H2O"]["Tb"], args=
(
    Comp["H2O"]["Tc"],
    Comp["H2O"]["omega"]
)
)

print(Hig)
print(HL_h2o)

Hr = Hres(40, toK(120), Comp["N2"]["Tc"],  Comp["N2"]["Pc"], Comp["N2"]["omega"] , False)
H = Hig[0] + HL_h2o[0]

print(f'{H/1000} kJ/mol')

(303496158.6672187, 3.3694842323763597e-06)
(4655.170180889767, 5.168277118372274e-11)
303500.8138373996 kJ/mol


In [81]:
import chemicals.heat_capacity as hc


water_low_gas_coeffs = [30.09200, 6.832514/1e3, 6.793435/1e6, -2.534480/1e9, 0.082139*1e6]
gi = hc.Shomate_integral(toK(100), *water_low_gas_coeffs) - hc.Shomate_integral(toK(25), *water_low_gas_coeffs)

print(integrate.quad(hc.Rowlinson_Bondi, 0, toK(100), (Comp["H2O"]["Tc"], Comp["H2O"]["omega"], gi))[0] / 1000)

1441.1022814253024


/tmp/ipykernel_6375/798857567.py:7: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  print(integrate.quad(hc.Rowlinson_Bondi, 0, toK(100), (Comp["H2O"]["Tc"], Comp["H2O"]["omega"], gi))[0] / 1000)
